# 55 Opintopistettä suorittaneet - esimerkki

Tämä notebook avaa 55 opintopistettä suorittaneiden tilastoja. 

Johtoajatuksena on esitellä miten dataa voisi antaa käpistellä hieman siten, että käyttäjän ei tarvitse itse koodata.

**Aja jokainen koodisolu valitsemalla se ja painamalla CTRL+Enter**

In [ ]:
# This is to demonstrate handling downloading a resource requiring simple token authentication

import getpass

password = getpass.getpass("please give password")

In [ ]:
from ipywidgets import interact
# download the data to a local file
# thias may take a while
from urllib.request import urlretrieve

destination = "/tmp/ytl_arvosanat.json"
#note we could use password here to authenticate outside the scope of this dashboard
url = "http://api.vipunen.fi/api/resources/suorittanut55/data"
_,_ = urlretrieve(url, destination)


In [ ]:

import pandas as pd
import json
with open(destination, "r") as file_:
    df = pd.DataFrame.from_records(json.load(file_))

df_original = df

## Interact

IPywidgetsin interact mahdollistaa interaktiivisten, koodaamattomien näkymien tekemisen.


In [ ]:
@interact(oppilaitos1=["Lappeenrannan teknillinen yliopisto", "Aalto-yliopisto", "Lahden ammattikorkeakoulu"],
         oppilaitos2=["Lappeenrannan teknillinen yliopisto", "Aalto-yliopisto", "Lahden ammattikorkeakoulu"],
         vuosi=[2018, 2017, 2016, 2015])
def compare(oppilaitos1, oppilaitos2, vuosi):
    to_compare = (oppilaitos1, oppilaitos2)
    one_year = df[df["vuosi"] == int(vuosi)]
    one_year[one_year["oppilaitos"].isin(to_compare)].groupby(["okmOhjauksenAla", "oppilaitos"])["suorittanut55"].sum().plot.barh(title="Kahden koulun vertailu")

On helppo huomata, että vertailu ilman normalisointia on merkityksetöntä: ei ole tiedossa montako opiskelijaa ylipäänsä olisi voinut saada 55op kussakin opinahjossa.

In [ ]:
@interact(oppilaitos=["Lappeenrannan teknillinen yliopisto", "Aalto-yliopisto", "Lahden ammattikorkeakoulu"])
def plot_it(oppilaitos):
    df[df["oppilaitos"] == oppilaitos]\
    .groupby(["vuosi"])["suorittanut55"].sum().plot.barh(title="Eri vuosien vertailu yhden koulun datasta")

## Merkkijono

Syöte voi olla myös vapaa merkkijono, jos halutaan tehdä jotain monimutkaisempaa.

In [ ]:
@interact(school="Aalto-yliopisto")
def one_degree_program(school):
    field = "Tietojenkäsittely- ja tietoliikenne ICT), monialaiset koulutusohjelmat "
    title = "Vuosivertailu {0}:n ohjelmasta {1}".format(school, field)

    df[(df["oppilaitos"] == school) 
             & (df_original["koulutusala(taso3)"] == field)
            ].groupby(["vuosi"])["suorittanut55"].sum().plot.barh(title=title)